In [2]:
import numpy as np
import pandas as pd

In [3]:
# Import the BMMS overview file (this file is pre edited in Excel)
BMMS_df = pd.read_excel('data/BMMS_overview_edited_v2.xls')
# Drop duplicates
BMMS_df = BMMS_df.drop_duplicates()
BMMS_df.head(2)

,road,km,type,LRPName,name,length,condition,structureNr,roadName,chainage,width,constructionYear,spans,zone,circle,division,sub-division,lat,lon,EstimatedLoc
0,Z7702,5.310,Baily with Steel Deck,LRP006a,.,12.4,A,109195,Signboard-Morelganj-Sharankhola-Rainda-Bogi Road,5.310,4.15,1988.0,1.0,Khulna,Khulna,Bagerhat,Bagerhat-1,32.561222,89.860361,bcs1
1,Z7606,3.163,Box Culvert,LRP003a,.,2.0,C,108705,Gollamari-Batiaghata-Dacope-Nalian Forest Road,3.163,7.35,1982.0,1.0,Khulna,Khulna,Khulna,Khulna-2,29.777083,89.534000,bcs1


After initial cleaning in excel, there were still several outliers in the Java programme. Some of the lon lat data was inconsistent with the rest of the bridges on the same road. This notebook compares the incorrect location of the bridge on road N1 and LRP456b as a reference to be compared to different structures within the same LRP that are in the correct location. 

In [5]:
# Check reference road (this one is correct)
BMMS_df.loc[BMMS_df['LRPName']=='LRP456a']

,road,km,type,LRPName,name,length,condition,structureNr,roadName,chainage,width,constructionYear,spans,zone,circle,division,sub-division,lat,lon,EstimatedLoc
13668,N1,459.681,Box Culvert,LRP456a,BUS STAND TEKNAF BOX CULVERT,1.50,A,100891,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,459.681,10.30,1991.0,1.0,Chittagong,Chittagong,Cox's Bazar,Cox's Bazar - 2,20.868944,92.298222,bcs1
15153,N5,453.198,Box Culvert,LRP456a,.,2.35,A,112013,Dhaka (Mirpur)-Utholi-Paturia- Natakhola-Kashi...,453.198,11.23,1991.0,1.0,Rangpur,Dinajpur,Panchgarh,Panchgarh,26.188167,88.538167,bcs1


In [6]:
# Locate a wrong bridge (note the 2 degrees difference in longitude). Coordinates are identified through the java programme
BMMS_df.loc[(BMMS_df['lon'] > 90.1000000) & (BMMS_df['lon'] < 91.7000000) & (BMMS_df['lat'] > 20.7000000) & (BMMS_df['lat'] < 21.7000000)]

,road,km,type,LRPName,name,length,condition,structureNr,roadName,chainage,width,constructionYear,spans,zone,circle,division,sub-division,lat,lon,EstimatedLoc
13669,N1,459.866,Box Culvert,LRP456b,BUS STAND TEKNAF BOX CULVERT,1.5,C,100893,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,459.866,10.3,1989.0,1.0,Chittagong,Chittagong,Cox's Bazar,Cox's Bazar - 2,20.867278,90.298389,bcs1


To fix the data, only the road, LRP, latitude, and longitude data are needed.

In [7]:
# Slice the data 
BMMS = BMMS_df.loc[:,['road', 'LRPName', 'lat', 'lon']]

In [9]:
# Sort values first on road second on LRP name in order to get reference points for wrong entries
BMMS.sort_values(['road','LRPName'], inplace = True)

In [10]:
BMMS

,road,LRPName,lat,lon
12885,N1,LRP001a,23.702889,90.450389
12886,N1,LRP004b,23.693611,90.478833
12887,N1,LRP008b,23.702083,90.515917
12888,N1,LRP008b,23.704583,90.518833
12889,N1,LRP010a,23.702056,90.528194
...,...,...,...,...
21291,Z8948,LRP001a,22.419167,90.787500
21292,Z8948,LRP002a,22.414167,90.791667
21293,Z8948,LRP002c,22.414167,90.791944
21294,Z8948,LRPSa,22.425278,90.787778


The values were sorted on road and LRP name so that for each road, the previous value would be geographically the nearest bridge. To identify wrong values, each longitude and latitude coordinate were compared to the values in the row above it (under the condition that they are on the same road). If the absolute difference between the two is greater than 0.3 (degrees), I assume that it is a wrong input as this would mean the structure is geographically far removed from the previous structure on the road.

The backfill method was used to fill in the wrong data, which means it will get the same longitude and/or latitude values as the previous entry. 

In [13]:
# Loop through both lon and lat and when the value on the same road is to different from the previous change the value to the previous entry
for i in range(1,21295): 
    if abs(BMMS.iloc[i,3] - BMMS.iloc[i-1,3]) > 0.3:
        if BMMS.iloc[i,0] == BMMS.iloc[i-1,0]:
            BMMS.iloc[i,3] = BMMS.iloc[i-1,3]
    if abs(BMMS.iloc[i,2] - BMMS.iloc[i-1,2]) > 0.3:
        if BMMS.iloc[i,0] == BMMS.iloc[i-1,0]:
            BMMS.iloc[i,2] = BMMS.iloc[i-1,2]

In [14]:
# Check our reference location and see the longitude is now updated
BMMS.loc[BMMS['LRPName']=='LRP456b']

,road,LRPName,lat,lon
13669,N1,LRP456b,20.867278,92.298222


In [18]:
# Add index column for merging
BMMS_df['index_df'] = BMMS_df.index

In [19]:
# Add index column for merging
BMMS['index'] = BMMS.index

In [20]:
# Merge back together
BMMS_export = BMMS_df.merge(BMMS[['index','lat','lon']], left_on ='index_df', right_on = 'index', how='left')

In [21]:
# Drop irrelevant coordinates
BMMS_export.drop(['lat_x', 'lon_x', 'index', 'index_df'], axis=1,inplace=True)

In [22]:
# Check reference row
BMMS_export.loc[BMMS_export['LRPName']=='LRP456b']

,road,km,type,LRPName,name,length,condition,structureNr,roadName,chainage,width,constructionYear,spans,zone,circle,division,sub-division,EstimatedLoc,lat_y,lon_y
13669,N1,459.866,Box Culvert,LRP456b,BUS STAND TEKNAF BOX CULVERT,1.5,C,100893,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,459.866,10.3,1989.0,1.0,Chittagong,Chittagong,Cox's Bazar,Cox's Bazar - 2,bcs1,20.867278,92.298222


In [23]:
# Rename to original names
BMMS_export.rename(columns={"lat_y": "lat", "lon_y": "lon"},inplace=True)

In [24]:
# Sort the values again
BMMS_export.sort_values(['road', 'LRPName'],inplace=True)

In [25]:
# Export
#BMMS_export.to_excel("BMMS_Overview_Python_editv2.xlsx")

Do note that the export file can not be plugged straight into the java programme. The estimated Loc column needs to be moved after the coordinates and the index column needs to be removed. 

If that does not work, copy all the column values and paste them over the data in the original java BMMS_overview 